В этом домашнем задании вы обучите GAN генерировать лица людей и посмотрите на то, как можно оценивать качество генерации

In [5]:
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as tt

from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from tqdm.notebook import tqdm

%matplotlib inline

In [6]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [8]:
!nvidia-smi

Wed Dec 21 15:04:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Часть 1. Подготовка данных (1 балл)

Так как старая ссылка на Google Drive с датасетом не работает, загрузим копию датасета [напрямую с Kaggle](https://www.kaggle.com/datasets/tommykamaz/faces-dataset-small)

In [9]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d tommykamaz/faces-dataset-small

100% 3.99G/3.99G [01:51<00:00, 42.2MB/s]
100% 3.99G/3.99G [01:51<00:00, 38.3MB/s]


In [11]:
!mkdir images
!unzip -q -o faces-dataset-small.zip -d images | awk 'BEGIN {ORS=" "} {if(NR%50==0)print "."}'

In [12]:
DATA_DIR = './images/'

Давайте загрузим наши изображения. Напишите функцию, которая строит DataLoader для изображений, при этом меняя их размер до нужного значения (размер 1024 слишком большой, поэтому мы рекомендуем взять размер 128 либо немного больше)

In [13]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [14]:
def get_dataloader(image_size, batch_size, device):
    """
    Builds dataloader for training data.
    Use tt.Compose and tt.Resize for transformations
    :param image_size: height and wdith of the image
    :param batch_size: batch_size of the dataloader
    :returns: DataLoader object 
    """

    train_ds = ImageFolder(DATA_DIR, transform=tt.Compose([
    tt.Resize(image_size),
    tt.CenterCrop(image_size),
    tt.ToTensor(),
    tt.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

    train_dl = DataLoader(train_ds, batch_size, shuffle=True, pin_memory=True)
    return train_ds, DeviceDataLoader(train_dl, device)

In [15]:
image_size = 128
batch_size = 16

train_dataset, train_loader = get_dataloader(image_size, batch_size, device)

In [17]:
first_batch = next(iter(train_loader))
plt.figure(figsize=(12, 12))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(make_grid(first_batch[0][:64], padding=10, normalize=True).cpu(), (1, 2, 0)));

## Часть 2. Построение и обучение модели (2 балла)

Сконструируйте генератор и дискриминатор. Помните, что:
* дискриминатор принимает на вход изображение (тензор размера `3 x image_size x image_size`) и выдает вероятность того, что изображение настоящее (тензор размера 1)

* генератор принимает на вход тензор шумов размера `latent_size x 1 x 1` и генерирует изображение размера `3 x image_size x image_size`

Возьмем за основу DCGAN из [официальной документации PyTorch](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html) 

In [31]:
num_channels = 3
discriminator_feature_maps = 32
generator_feature_maps = 64

In [32]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [33]:
discriminator = nn.Sequential(
    # input is num_channels x 128 x 128
    nn.Conv2d(num_channels, discriminator_feature_maps, kernel_size=4, stride=2, padding=1, bias=False),
    nn.LeakyReLU(0.2, inplace=True),

    # state size. discriminator_feature_maps x 64 x 64
    nn.Conv2d(discriminator_feature_maps, discriminator_feature_maps * 2, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(discriminator_feature_maps * 2),
    nn.LeakyReLU(0.2, inplace=True),

    # state size. (discriminator_feature_maps * 2) x 32 x 32
    nn.Conv2d(discriminator_feature_maps * 2, discriminator_feature_maps * 4, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(discriminator_feature_maps * 4),
    nn.LeakyReLU(0.2, inplace=True),

    # state size. (discriminator_feature_maps * 4) x 16 x 16
    nn.Conv2d(discriminator_feature_maps * 4, discriminator_feature_maps * 8, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(discriminator_feature_maps * 8),
    nn.LeakyReLU(0.2, inplace=True),

    # state size. (discriminator_feature_maps * 8) x 8 x 8
    nn.Conv2d(discriminator_feature_maps * 8, discriminator_feature_maps * 16, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(discriminator_feature_maps * 16),
    nn.LeakyReLU(0.2, inplace=True),

    # state size. (discriminator_feature_maps * 16) x 4 x 4
    nn.Conv2d(discriminator_feature_maps * 16, 1, kernel_size=4, stride=1, padding=0, bias=False),

    # state size. 1 x 1 x 1
    nn.Flatten(),
    nn.Sigmoid()
)

In [34]:
discriminator.to(device)

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2.
discriminator.apply(weights_init)

# Print the model
print(discriminator)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (1): LeakyReLU(negative_slope=0.2, inplace=True)
  (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2, inplace=True)
  (5): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2, inplace=True)
  (8): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): LeakyReLU(negative_slope=0.2, inplace=True)
  (11): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (12): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
 

In [35]:
latent_size = 128

generator = nn.Sequential(
    # input is latent_size, going into a convolution
    nn.ConvTranspose2d(latent_size, generator_feature_maps * 16, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(generator_feature_maps * 16),
    nn.ReLU(True),

    # state size. (generator_feature_maps * 16) x 4 x 4
    nn.ConvTranspose2d(generator_feature_maps * 16, generator_feature_maps * 8, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(generator_feature_maps * 8),
    nn.ReLU(True),

    # state size. (generator_feature_maps * 8) x 8 x 8
    nn.ConvTranspose2d(generator_feature_maps * 8, generator_feature_maps * 4, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(generator_feature_maps * 4),
    nn.ReLU(True),

    # state size. (generator_feature_maps * 4) x 16 x 16
    nn.ConvTranspose2d(generator_feature_maps * 4, generator_feature_maps * 2, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(generator_feature_maps * 2),
    nn.ReLU(True),

    # state size. (generator_feature_maps * 2) x 32 x 32
    nn.ConvTranspose2d(generator_feature_maps * 2, generator_feature_maps, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(generator_feature_maps),
    nn.ReLU(True),

    # state size. generator_feature_maps x 64 x 64
    nn.ConvTranspose2d(generator_feature_maps, num_channels, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # state size. num_channels x 64 x 64
)

In [36]:
generator.to(device)

# Apply the weights_init function to randomly initialize all weights to mean=0, stdev=0.2.
generator.apply(weights_init)

# Print the model
print(generator)

Sequential(
  (0): ConvTranspose2d(128, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (10): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU(inplace=True)
  (12): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

Перейдем теперь к обучению нашего GANа. Алгоритм обучения следующий:
1. Учим дискриминатор:
  * берем реальные изображения и присваиваем им метку 1
  * генерируем изображения генератором и присваиваем им метку 0
  * обучаем классификатор на два класса

2. Учим генератор:
  * генерируем изображения генератором и присваиваем им метку 0
  * предсказываем дискриминаторором, реальное это изображение или нет


В качестве функции потерь берем бинарную кросс-энтропию

In [37]:
def fit(train_dl, model, optimizer, criterion, epochs):
    model["discriminator"].train()
    model["generator"].train()
    torch.cuda.empty_cache()
    
    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
    
    for epoch in range(epochs):
        loss_d_per_epoch = []
        loss_g_per_epoch = []
        real_score_per_epoch = []
        fake_score_per_epoch = []
        for real_images, _ in tqdm(train_dl):
            # Train discriminator
            # Clear discriminator gradients
            optimizer["discriminator"].zero_grad()
            # Pass real images through discriminator
            real_preds = model["discriminator"](real_images)
            real_targets = torch.ones(real_images.size(0), 1, device=device)
            real_loss = criterion["discriminator"](real_preds, real_targets)
            cur_real_score = torch.mean(real_preds).item()
            
            # Generate fake images
            latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
            fake_images = model["generator"](latent)

            # Pass fake images through discriminator
            fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
            fake_preds = model["discriminator"](fake_images)
            fake_loss = criterion["discriminator"](fake_preds, fake_targets)
            cur_fake_score = torch.mean(fake_preds).item()

            real_score_per_epoch.append(cur_real_score)
            fake_score_per_epoch.append(cur_fake_score)

            # Update discriminator weights
            loss_d = real_loss + fake_loss
            loss_d.backward()
            optimizer["discriminator"].step()
            loss_d_per_epoch.append(loss_d.item())


            # Train generator
            # Clear generator gradients
            optimizer["generator"].zero_grad()
            
            # Generate fake images
            latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
            fake_images = model["generator"](latent)
            
            # Try to fool the discriminator
            preds = model["discriminator"](fake_images)
            targets = torch.ones(batch_size, 1, device=device)
            loss_g = criterion["generator"](preds, targets)
            
            # Update generator weights
            loss_g.backward()
            optimizer["generator"].step()
            loss_g_per_epoch.append(loss_g.item())
            
        # Record losses & scores
        losses_g.append(np.mean(loss_g_per_epoch))
        losses_d.append(np.mean(loss_d_per_epoch))
        real_scores.append(np.mean(real_score_per_epoch))
        fake_scores.append(np.mean(fake_score_per_epoch))
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch + 1, epochs, 
            losses_g[-1], losses_d[-1], real_scores[-1], fake_scores[-1]))
    
    return losses_g, losses_d, real_scores, fake_scores

In [38]:
lr = 0.0002
max_epochs = 10

model = {
    "discriminator": discriminator,
    "generator": generator
}

optimizer = {
    "discriminator": optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999)),
    "generator": optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
}

criterion = {
    "discriminator": nn.BCELoss(),
    "generator": nn.BCELoss()
}

losses_g, losses_d, real_scores, fake_scores = fit(train_loader, model, optimizer, criterion, max_epochs)

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/10], loss_g: 13.3069, loss_d: 1.3599, real_score: 0.8088, fake_score: 0.3102


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [2/10], loss_g: 15.3513, loss_d: 1.0825, real_score: 0.8314, fake_score: 0.2168


  0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch [8/10], loss_g: 2.7135, loss_d: 0.9887, real_score: 0.6940, fake_score: 0.3110


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [9/10], loss_g: 2.8286, loss_d: 1.0367, real_score: 0.6645, fake_score: 0.3094


  0%|          | 0/50 [00:00<?, ?it/s]

Epoch [10/10], loss_g: 3.2158, loss_d: 0.8353, real_score: 0.7291, fake_score: 0.2652


Постройте графики лосса для генератора и дискриминатора. Что вы можете сказать про эти графики?

## Часть 3. Генерация изображений (1 балл)

Теперь давайте оценим качество получившихся изображений. Напишите функцию, которая выводит изображения, сгенерированные нашим генератором

In [47]:
n_images = 32

fixed_latent = torch.randn(n_images, latent_size, 1, 1, device=device)
model["generator"].eval()
with torch.no_grad():
    fake_images = model["generator"](fixed_latent).cpu()

In [48]:
def show_images(generated):
    plt.figure(figsize=(20, 20))
    plt.axis("off")
    plt.title("Fake Images")
    plt.imshow(np.transpose(make_grid(fake_images, padding=5, normalize=True),(1, 2, 0)))

In [49]:
show_images(fake_images)

Как вам качество получившихся изображений?

## Часть 4. Leave-one-out-1-NN classifier accuracy (6 баллов)

### 4.1. Подсчет accuracy (4 балла)

Не всегда бывает удобно оценивать качество сгенерированных картинок глазами. В качестве альтернативы вам предлагается реализовать следующий подход:
  * Сгенерировать столько же фейковых изображений, сколько есть настоящих в обучающей выборке. Присвоить фейковым метку класса 0, настоящим – 1.
  * Построить leave-one-out оценку: обучить 1NN Classifier (`sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)`) предсказывать класс на всех объектах, кроме одного, проверить качество (accuracy) на оставшемся объекте. В этом вам поможет `sklearn.model_selection.LeaveOneOut`

In [50]:
# на всем датасете LeaveOneOut работает слишком долго, поэтому возьмем небольшую подвыборку
n_images = 500
assert n_images < len(train_dataset)

In [53]:
fixed_latent = torch.randn(n_images, latent_size, 1, 1, device=device)

model["generator"].eval()
with torch.no_grad():
    fake_images = model["generator"](fixed_latent).cpu()

In [52]:
real_images = []
for idx in tqdm(range(n_images)):
    real_images.append(train_dataset[idx][0])

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
fake_images_reshape = fake_images.reshape(fake_images.shape[0], -1)
real_images_reshape = torch.sta

Что вы можете сказать о получившемся результате? Какой accuracy мы хотели бы получить и почему?

### 4.2. Визуализация распределений (2 балла)

Давайте посмотрим на то, насколько похожи распределения настоящих и фейковых изображений. Для этого воспользуйтесь методом, снижающим размерность (к примеру, TSNE) и изобразите на графике разным цветом точки, соответствующие реальным и сгенерированным изображенияи

Прокомментируйте получившийся результат: